<a href="https://colab.research.google.com/github/BharatXBhalla/Data-Analytics-CDAC/blob/main/Predictive_Model/Pyspark_ML_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark
import pyspark 
from pyspark.sql import SparkSession 

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 46.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0102d300639a6a1d8dea3923e6f6a2a4523e0d0dc1d6fbfb4586a9ca2c208fc5
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
spark=SparkSession.builder.appName('housing_price_model').getOrCreate() 

In [4]:
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True) 
df.show(10) 

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:

df.columns 

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [6]:
from pyspark.ml.feature import StringIndexer 
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat') 
indexed=indexer.fit(df).transform(df)

In [7]:
for item in indexed.head(5): 
    print(item) 
    print('\n') 

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [8]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler 

In [10]:
assembler=VectorAssembler(inputCols=['Age', 
'Tonnage', 
'passengers', 
'length', 
'cabins', 
'passenger_density', 
'cruise_cat'],outputCol='features') 
output=assembler.transform(indexed) 
output.select('features','crew').show(5) 
output.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|     

In [11]:
final_data=output.select('features','crew') 

In [12]:
#splitting data into train and test 
train_data,test_data=final_data.randomSplit([0.7,0.3]) 
train_data.describe().show()

test_data.describe().show() 

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               115|
|   mean| 7.749652173913057|
| stddev|3.3555207012029253|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                43|
|   mean| 7.913255813953488|
| stddev|3.9117063144034687|
|    min|               0.6|
|    max|              21.0|
+-------+------------------+



In [13]:
from pyspark.ml.regression import LinearRegression 

#creating an object of class LinearRegression 
#object takes features and label as input arguments 

ship_lr=LinearRegression(featuresCol='features',labelCol='crew') 

In [14]:
trained_ship_model=ship_lr.fit(train_data) 

In [15]:
ship_results=trained_ship_model.evaluate(train_data) 

print('Rsquared Error :',ship_results.r2) 

Rsquared Error : 0.9104668383032207


In [16]:
unlabeled_data=test_data.select('features') 
unlabeled_data.show(5) 

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,122.0,28.5,1...|
|[5.0,133.5,39.59,...|
|[5.0,160.0,36.34,...|
|[8.0,110.0,29.74,...|
+--------------------+
only showing top 5 rows



In [17]:

predictions=trained_ship_model.transform(unlabeled_data) 
predictions.show() 

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|[4.0,220.0,54.0,1...|  21.542376448371854|
|[5.0,122.0,28.5,1...|  3.1868982102067958|
|[5.0,133.5,39.59,...|  12.940629725650158|
|[5.0,160.0,36.34,...|  15.058637044280509|
|[8.0,110.0,29.74,...|  12.595001869356041|
|[9.0,59.058,17.0,...|   7.781542984409818|
|[9.0,88.5,21.24,9...|   9.548280685790749|
|[9.0,113.0,26.74,...|  11.378004405741788|
|[9.0,116.0,26.0,9...|  11.032819944072845|
|[10.0,46.0,7.0,6....|   2.024421999512041|
|[10.0,81.76899999...|   8.651479355509544|
|[10.0,105.0,27.2,...|  11.556230696201144|
|[10.0,151.4,26.2,...|     9.6812076024578|
|[11.0,58.6,15.66,...|   7.498555640273398|
|[11.0,91.62700000...|   8.963840886863997|
|[12.0,2.329,0.94,...|-0.08938277029991326|
|[12.0,88.5,21.24,...|   9.451122588156444|
|[12.0,88.5,21.24,...|  10.773157584202501|
|[12.0,91.0,20.32,...|   8.916564365964168|
|[12.0,108.865,27....|  10.77703